In [32]:
import torch, torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from contextlib import contextmanager

from oil.cnnTrainer import CnnTrainer
from oil.datasets import CIFAR10, C10augLayers
from oil.networkparts import layer13
from oil.schedules import cosLr
from oil.utils import to_gpu
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
train_epochs = 150
net_config =        {'numClasses':10}
opt_config =        {'lr':.1, 'momentum':.9, 'weight_decay':1e-4, 'nesterov':True}
sched_config =      {'cycle_length':train_epochs,'cycle_mult':1}
trainer_config =    {'amntLab':1, 'amntDev':5000,'dataseed':0,
                    'lab_BS':50, 'num_workers':4, 'log':False, 
                    }
trainer_config['description'] = "13Layer network, {} dev".format(trainer_config['amntDev'])
savedir = None#'/home/maf388/tb-experiments/layer13dev/'

def makeTrainer():
    CNN = layer13(**net_config)
    fullCNN = nn.Sequential(C10augLayers(),CNN)
    datasets = CIFAR10(aug=False)
    opt_constr = lambda params: optim.SGD(params, **opt_config)
    lr_lambda = cosLr(**sched_config)
    return CnnTrainer(fullCNN, datasets, opt_constr, lr_lambda, **trainer_config)

In [3]:
trainer = makeTrainer()
trainer.load_checkpoint('/home/maf388/tb-experiments/layer13dev/checkpoints/c.150.ckpt')
_ = trainer.CNN.eval()

Files already downloaded and verified
Files already downloaded and verified
Creating Train, Dev split         with 45000 Train and 5000 Dev
=> loading checkpoint '/home/maf388/tb-experiments/layer13dev/checkpoints/c.150.ckpt'


In [4]:
trainer.getAccuracy(trainer.dev)

0.9442

In [9]:
def fdFvpBatch(trainer, vec, trainData, eps=1, stencil=[-1,1]):
    vlist = unflatten_like(vec, trainer.CNN.parameters())
    derivLogP = 0
    for i, c in enumerate(stencil):
        with add(net, vlist, mul=eps*(i-(len(stencil)-1)/2)):
            logits = trainer.CNN(*traindata)
            logPk = nn.LogSoftmax(dim=1)(logits)
            derivLogP += c*logPk.detach()/eps
    with torch.autograd.enable_grad():
        logits = trainer.CNN(*traindata)
        Pk = nn.Softmax(dim=1)(logits)
        score = torch.dot(Pk, derivLogP)
        Fvp_list = torch.aurograd.grad(score, trainer.CNN.parameters())
    return flatten(Fvp_list)

In [10]:
def autoFvpBatch(trainer,vec, trainData):
    

SyntaxError: invalid syntax (<ipython-input-10-f833bef3d394>, line 1)

In [31]:
#Simple example sanity check
class Toy(nn.Module):
    def __init__(self, a1, a2):
        super().__init__()
        self.a1 = a1
        self.a2 = a2
    def forward(self, A1, A2):
        return F.bilinear(self.a1,self.a1,A1)+F.bilinear(self.a2,self.a2,A2)
        
class ToyTrainer(object):
    def __init__(self):
        self.A1 = torch.eye(5)
        self.A1[2,2]=3
        self.A1[4,4]=-1
        m = torch.normal(torch.zeros(5,5))
        _,rot = torch.symeig(m.t()+m,eigenvectors=True)
        self.A1 = torch.mm(rot, self.A1)
        self.A2 = torch.eye(10)
        self.CNN = Toy(torch.zeros(5),torch.ones(10))
        
    def loss(self, *trainData):
        return self.CNN(self.A1, self.A2)
    